<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Векторизация" data-toc-modified-id="Векторизация-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Векторизация</a></span></li><li><span><a href="#Разделение-на-тестовую-и-обучающую-выборки" data-toc-modified-id="Разделение-на-тестовую-и-обучающую-выборки-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Разделение на тестовую и обучающую выборки</a></span></li><li><span><a href="#Разделение-на-тестовую-и-обучающую-выборки" data-toc-modified-id="Разделение-на-тестовую-и-обучающую-выборки-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Разделение на тестовую и обучающую выборки</a></span></li><li><span><a href="#Векторизация" data-toc-modified-id="Векторизация-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Векторизация</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import OrdinalEncoder

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score


from scipy import stats as st

random = 12345

import warnings
warnings.filterwarnings('ignore')

import time

from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.corpus import stopwords
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

import spacy #load spacy
from tqdm.notebook import tqdm

In [24]:
toxic_comments = pd.read_csv('toxic_comments (1).csv')

In [25]:
toxic_comments

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


In [26]:
display(toxic_comments.head(5))
display(toxic_comments.info())
display(toxic_comments.describe())

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


None

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


Судя по среднему 0.01 у нас наблюдается дисбаланс классов, что надо иметь ввиду в случае неудовлетворительных результатов.

### Лемматизация

In [27]:
def clear_text(row):
    text = row['text']
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\']', ' ', text)
    text = text.split()
    text = " ".join(text)    
    return text

In [28]:
toxic_comments['clear_text'] = toxic_comments.apply(clear_text, axis=1)

In [29]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [30]:
def lematization(row):
    text = row['clear_text']
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    
    return ' '.join(tokens)

In [31]:
toxic_comments['text_spacy'] = toxic_comments.apply(lematization, axis=1)

In [32]:
toxic_comments.head(5)

,Unnamed: 0,text,toxic,clear_text,text_spacy
0,0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...,explanation edit username hardcore metallica f...
1,1,D'aww! He matches this background colour I'm s...,0,d'aww he matches this background colour i'm se...,d'aww match background colour seemingly stick ...
2,2,"Hey man, I'm really not trying to edit war. It...",0,hey man i'm really not trying to edit war it's...,hey man try edit war guy constantly remove rel...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,more i can't make any real suggestions on impr...,real suggestion improvement wonder section sta...
4,4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...,sir hero chance remember page


### Разделение на тестовую и обучающую выборки

In [33]:
features = toxic_comments['text_spacy']
target = toxic_comments['toxic']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=0)

In [35]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((127433,), (31859,), (127433,), (31859,))

### Векторизация

In [36]:
tfidfconverter = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.7, stop_words=stopwords.words('russian'))

X_train = tfidfconverter.fit_transform(X_train.values)
X_test = tfidfconverter.transform(X_test.values)

### Вывод

- Библиотеки загружены;
- Данные загружены;
- Данные осмотрены;
    - Наблюдается дисбаланс классов (при неуд. результатах работы модели, сбалансировать классы);
- Данные подготовлены;
    - Проведена лемматизация;
    - Проведена векторизация;
    - Проведен раздел на обучающую и тестовую выборки.

## Обучение

In [37]:
variable = ['F1', 'T_train', 'T_predict']
model = ['RandomForestRegressor', 'LGBMRegressor', 'XGBRegressor']
F1 = []
training_time = []
prediction_time = []

In [38]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12345)

param_grid = {'n_estimators': [int(x) for x in np.linspace(start = 10, stop = 110, num = 30)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [6, 10, 24],
               'min_samples_split': [2, 5],
               'min_samples_leaf': [1, 2],
               'bootstrap': [True, False]}

rf_RandomGrid = RandomizedSearchCV(estimator = RandomForestClassifier(random_state=12345)
                                   , param_distributions = param_grid
                                   , cv = skf
                                   , verbose=0
                                   , n_jobs = -1
                                   , scoring="f1"
                                   , random_state = 12345)
  
rf_RandomGrid.fit(X_train, y_train)

print('-----------------------')
print('Лучшие параметры модели:')
print(rf_RandomGrid.best_estimator_)
print('')
print('F1:')
print(rf_RandomGrid.best_score_)
print('-----------------------')

best_rf_classifier = RandomForestClassifier(n_estimators = rf_RandomGrid.best_params_["n_estimators"],
                            max_features      = rf_RandomGrid.best_params_["max_features"],
                            max_depth         = rf_RandomGrid.best_params_["max_depth"],
                            min_samples_split = rf_RandomGrid.best_params_["min_samples_split"],
                            min_samples_leaf  = rf_RandomGrid.best_params_["min_samples_leaf"],
                            bootstrap         = rf_RandomGrid.best_params_["bootstrap"])

t0 = time.time()
best_rf_classifier.fit(X_train, y_train)
t1 = time.time()

t2 = time.time()
predictions = best_rf_classifier.predict(X_test)
t3 = time.time()

#Так можно вывести результаты всех итераций
#cv_results = rf_RandomGrid.cv_results_
#for mean_score, params in zip(cv_results["mean_test_score"], cv_results["params"]):
#    print(np.sqrt(-mean_score), params)

result = f1_score(y_test, predictions)
print("F1 модели RandomForestClassifier на валидационной выборке:", result)

print('Затраченное время на обучение модели:', t1-t0)
print('Затраченное время предсказания модели:', t3-t2)

F1.append(result)
training_time.append(t1-t0)
prediction_time.append(t3-t2)

/Users/kirilligorevich/opt/anaconda3/envs/sd/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/kirilligorevich/opt/anaconda3/envs/sd/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/kirilligorevich/opt/anaconda3/envs/sd/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, expli

-----------------------
Лучшие параметры модели:
RandomForestClassifier(max_depth=24, max_features='auto', min_samples_leaf=2,
                       n_estimators=30, random_state=12345)

F1:
0.47746120590849966
-----------------------
F1 модели линейной регрессии на валидационной выборке: 0.4835932045613218
Затраченное время на обучение модели: 4.3478310108184814
Затраченное время предсказания модели: 0.12276196479797363


In [39]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12345)

params = {
    'num_leaves': [7, 100],
    'objective': ['regression','binary'],
    'n_estimators': [1, 100],
}

lgbm_RandomGrid = RandomizedSearchCV(lgb.LGBMClassifier(random_state=12345), 
                                     params, 
                                     verbose=0,
                                     n_jobs = -1,
                                     scoring='f1',
                                     cv=skf,
                                     random_state = 12345)

lgbm_RandomGrid.fit(X_train, y_train)


print('-----------------------')
print('Лучшие параметры модели:')
print(lgbm_RandomGrid.best_estimator_)
print('')
print('F1:')
print(lgbm_RandomGrid.best_score_)
print('-----------------------')

best_lgbm_classifier = lgb.LGBMClassifier(num_leaves = lgbm_RandomGrid.best_params_["num_leaves"],
                           n_estimators  = lgbm_RandomGrid.best_params_["n_estimators"],
                           objective     = lgbm_RandomGrid.best_params_["objective"],
                           scoring='f1')

t0 = time.time()
best_lgbm_classifier.fit(X_train, y_train)
t1 = time.time()

t2 = time.time()
predictions = best_lgbm_classifier.predict(X_test)
t3 = time.time()

result = f1_score(y_test, predictions) 
print("F1 модели LGBMClassifier на валидационной выборке:", result)

print('Затраченное время на обучение модели:', t1-t0)
print('Затраченное время на предсказание модели:', t3-t2)

#F1.append(result)
#training_time.append(t1-t0)
#prediction_time.append(t3-t2)

-----------------------
Лучшие параметры модели:
LGBMClassifier(num_leaves=100, objective='binary', random_state=12345)

F1:
0.745700292128544
-----------------------
F1 модели линейной регрессии на валидационной выборке: 0.7492907801418439
Затраченное время на обучение модели: 25.709641933441162
Затраченное время на предсказание модели: 0.5894439220428467


Подготовка данных для xgboost (без урезания данных считалось слишком долго)

In [40]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12345)

param_grid = {"max_depth":    [10, 30],
              "learning_rate": [0.01, 0.1]}


XGBR_RandomGrid = RandomizedSearchCV(xgb.XGBClassifier(scoring='f1', n_estimators = 50, random_state = 12345),
                                     param_grid,
                                     verbose=0,
                                     cv=skf,
                                     #n_jobs = -1,
                                     random_state=12345)

XGBR_RandomGrid.fit(X_train, y_train)

print('-----------------------')
print('Лучшие параметры модели:')
print(XGBR_RandomGrid.best_estimator_)
print('')
print('F1:')
print(XGBR_RandomGrid.best_score_)
print('-----------------------')

best_xgbr_classifier = xgb.XGBClassifier(learning_rate = XGBR_RandomGrid.best_params_["learning_rate"],
                           n_estimators  = 50,
                           max_depth     = XGBR_RandomGrid.best_params_["max_depth"],
                           scoring='f1')

t0 = time.time()
best_xgbr_classifier.fit(X_train, y_train)
t1 = time.time()

t2 = time.time()
predictions = best_xgbr_classifier.predict(X_test)
t3 = time.time()

result = f1_score(y_test, predictions) 
print("F1 модели XGBClassifier на валидационной выборке:", result)

print('Затраченное время на обучение модели:', t1-t0)
print('Затраченное время на предсказание модели:', t3-t2)

#F1.append(result)
#training_time.append(t1-t0)
#prediction_time.append(t3-t2)

[14:13:07] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1662013913324/work/src/learner.cc:627: 
Parameters: { "scoring" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:13:34] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1662013913324/work/src/learner.cc:627: 
Parameters: { "scoring" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:14:02] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1662013913324/work/src/learner.cc:627: 
Parameters: { "scoring" } might not be used.


In [41]:
best_rf_classifier = RandomForestClassifier(n_estimators = 13,
                            max_features      = 'sqrt',
                            max_depth         = 24,
                            min_samples_split = 2,
                            min_samples_leaf  = 2,
                            bootstrap         = False)

t0 = time.time()
best_rf_classifier.fit(X_train, y_train)
t1 = time.time()

t2 = time.time()
predictions = best_rf_classifier.predict(X_test)
t3 = time.time()

#Так можно вывести результаты всех итераций
#cv_results = rf_RandomGrid.cv_results_
#for mean_score, params in zip(cv_results["mean_test_score"], cv_results["params"]):
#    print(np.sqrt(-mean_score), params)

result = f1_score(y_test, predictions)
print("F1 модели RandomForestClassifier на валидационной выборке:", result)

print('Затраченное время на обучение модели:', t1-t0)
print('Затраченное время предсказания модели:', t3-t2)

#F1.append(result)
#training_time.append(t1-t0)
#prediction_time.append(t3-t2)

F1 модели линейной регрессии на валидационной выборке: 0.45961265942371277
Затраченное время на обучение модели: 4.183363914489746
Затраченное время предсказания модели: 0.07943320274353027


In [42]:
best_lgbm_classifier = lgb.LGBMClassifier(num_leaves = 100,
                           n_estimators  = 100,
                           objective     = 'binary',
                           scoring='f1')

t0 = time.time()
best_lgbm_classifier.fit(X_train, y_train)
t1 = time.time()

t2 = time.time()
predictions = best_lgbm_classifier.predict(X_test)
t3 = time.time()

result = f1_score(y_test, predictions) 
print("F1 модели LGBMClassifier на валидационной выборке:", result)

print('Затраченное время на обучение модели:', t1-t0)
print('Затраченное время на предсказание модели:', t3-t2)

F1.append(result)
training_time.append(t1-t0)
prediction_time.append(t3-t2)

[LightGBM] [Warning] Unknown parameter: scoring
F1 модели линейной регрессии на валидационной выборке: 0.7492907801418439
Затраченное время на обучение модели: 33.645912885665894
Затраченное время на предсказание модели: 0.7104558944702148


In [43]:
best_xgbr_classifier = xgb.XGBClassifier(learning_rate = XGBR_RandomGrid.best_params_["learning_rate"],
                           n_estimators  = 50,
                           max_depth     = XGBR_RandomGrid.best_params_["max_depth"],
                           scoring='f1')

t0 = time.time()
best_xgbr_classifier.fit(X_train, y_train)
t1 = time.time()

t2 = time.time()
predictions = best_xgbr_classifier.predict(X_test)
t3 = time.time()

result = f1_score(y_test, predictions) 
print("F1 модели XGBClassifier на валидационной выборке:", result)

print('Затраченное время на обучение модели:', t1-t0)
print('Затраченное время на предсказание модели:', t3-t2)

F1.append(result)
training_time.append(t1-t0)
prediction_time.append(t3-t2)

[14:36:05] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1662013913324/work/src/learner.cc:627: 
Parameters: { "scoring" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


F1 модели линейной регрессии на валидационной выборке: 0.7031602708803611
Затраченное время на обучение модели: 127.11199903488159
Затраченное время на предсказание модели: 0.1878190040588379


### Вывод:

**Модель RandomForestRegressor:**<br>
С помощью модели RandomForestRegressor получили лучший результат на тренировочной выборке F1 - 0.45 на валидационной выборке F1 - 0.49 на следующих параметрах:
- n_estimators - 13;
- min_samples_split - 2;
- min_samples_leaf - 2;
- max_features - sqrt;
- max_depth - 24;
- bootstrap - False.

Затраченное время:
- На обучение модели: 3 с;
- На предсказание: 0.1 с.

**Модель LightGBM:**<br>
С помощью модели lightgbm получили лучший результат на тренировочной выборке F1 - 0.75 на валидационной выборке F1 - 0.76 на следующих параметрах:
- objective - 'binary';
- num_leaves - 100;
- n_estimators - 100.

Затраченное время:
- На нахождение лучшей модели: 86 с;
- На предсказание: 1.9 с.

**Модель XGBClassifier:**<br>
С помощью модели XGBClassifier получили лучший результат на тренировочной выборке F1 - 0.95 на валидационной выборке F1 - 0.72 на следующих параметрах:
- max_depth - 30;
- learning_rate - 0.1;
- n_estimators - 50.

Затраченное время:
- На нахождение лучшей модели: 144 с;
- На предсказание: 0.7 с.

In [44]:
res_df = pd.DataFrame(np.column_stack([F1, training_time, prediction_time]), 
                               columns=variable, index=model)
display(res_df)

,F1,T_train,T_predict
RandomForestRegressor,0.483593,4.347831,0.122762
LGBMRegressor,0.749291,33.645913,0.710456
XGBRegressor,0.703160,127.111999,0.187819


## Выводы

- Библиотеки загружены;
- Данные загружены;
- Данные осмотрены;
    - Наблюдается дисбаланс классов (при неуд. результатах работы модели, сбалансировать классы);
- Данные подготовлены;
    - Проведена лемматизация;
    - Проведена векторизация;
    - Проведен раздел на обучающую и тестовую выборки.
    
**Обучены модели RandomForestRegressor, LightGBM, XGBRegressor и проведен их анализ**<br>
**Модель RandomForestRegressor:**<br>
С помощью модели RandomForestRegressor получили лучший результат на тренировочной выборке F1 - 0.45 на валидационной выборке F1 - 0.49 на следующих параметрах:
- n_estimators - 13;
- min_samples_split - 2;
- min_samples_leaf - 2;
- max_features - sqrt;
- max_depth - 24;
- bootstrap - False.

Затраченное время:
- На обучение модели: 3 с;
- На предсказание: 0.1 с.

**Модель LightGBM:**<br>
С помощью модели lightgbm получили лучший результат на тренировочной выборке F1 - 0.75 на валидационной выборке F1 - 0.76 на следующих параметрах:
- objective - 'binary';
- num_leaves - 100;
- n_estimators - 100.

Затраченное время:
- На нахождение лучшей модели: 86 с;
- На предсказание: 1.9 с.

**Модель XGBClassifier:**<br>
С помощью модели XGBClassifier получили лучший результат на тренировочной выборке F1 - 0.95 на валидационной выборке F1 - 0.72 на следующих параметрах:
- max_depth - 30;
- learning_rate - 0.1;
- n_estimators - 50.

Затраченное время:
- На нахождение лучшей модели: 144 с;
- На предсказание: 0.7 с.

**Выбор модели останавливаем на LightGBM, так как она показала лучшее значение F1 предсказания.**

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны